In [1]:
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
from collections import defaultdict
import re

from src.datasets.who_is_who import WhoIsWhoDataset
from src.shared.database_wrapper import DatabaseWrapper
from src.shared.graph_schema import NodeType, EdgeType
from src.shared.graph_sampling import GraphSampling
from src.shared import config

/home/vincie/.anaconda3/envs/master/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Configuration

In [2]:
max_iterations = 10000

In [3]:
db = DatabaseWrapper(database='homogeneous-graph')
#db.delete_all_nodes()

2024-10-10 15:00:47,145 - DatabaseWrapper - INFO - Connecting to the database ...
2024-10-10 15:00:47,145 - DatabaseWrapper - INFO - Database ready.


In [4]:
data = WhoIsWhoDataset.parse_data()
train_data = WhoIsWhoDataset.parse_train()

In [16]:
model = SentenceTransformer(
    'sentence-transformers/all-MiniLM-L6-v2',
    device='cuda'
)
print(f"Model dim: {model.get_sentence_embedding_dimension()}")

Model dim: 384


## Add publication nodes to the graph database

In [ ]:
def process_batch(batch):
    if not batch[NodeType.PUBLICATION]:
        return
    title_embs = model.encode(
        [node['title'] for node in batch[NodeType.PUBLICATION]]
    )
    abstract_embs = model.encode(
        [node['abstract'] for node in batch[NodeType.PUBLICATION]]
    )
    venue_embs = model.encode(
        [node['venue'] for node in batch[NodeType.PUBLICATION]]
    )
    for i, node in enumerate(batch[NodeType.PUBLICATION]):
        node['title_emb'] = title_embs[i]
        node['abstract_emb'] = abstract_embs[i]
        node['venue_emb'] = venue_embs[i]
    db.merge_nodes(NodeType.PUBLICATION, batch[NodeType.PUBLICATION])
    batch[NodeType.PUBLICATION] = []

In [18]:
batch_nodes = defaultdict(list)
current_iteration = 0

with tqdm(total=max_iterations) as pbar:
    for author_id, values in train_data.items():
        papers = values.get('normal_data', [])
        papers.extend(values.get('outliers', []))
        
        if max_iterations is not None and current_iteration >= max_iterations:
            break
        current_iteration += len(papers)
        
        for paper_id in papers:
            values = data[paper_id]
            paper_node = {
                'id': values['id'],
                'title': values['title'],
                'abstract': values['abstract'],
                'year': values['year'],
                'venue': values['venue'],
            }
            batch_nodes[NodeType.PUBLICATION].append(paper_node)
            
            if len(batch_nodes[NodeType.PUBLICATION]) % 1000 == 0:
                process_batch(batch_nodes)
                
            pbar.update(1)
        
process_batch(batch_nodes)

  0%|          | 0/10000 [00:00<?, ?it/s]

## Add true author data to the nodes

In [19]:
def reverse_dict(author_dict):
    paper_to_author = {}
    for author_id, values in author_dict.items():
        normal_papers = values.get('normal_data', [])
        for paper_id in normal_papers:
            paper_to_author[paper_id] = author_id
    return paper_to_author

paper_id_to_author = reverse_dict(train_data)

props = []
with tqdm(total=db.count_nodes(NodeType.PUBLICATION), desc="Merging WhoIsWho train_author.json") as pbar:
    for nodes in db.iter_nodes(NodeType.PUBLICATION, ['id']):
        for node in nodes:
            true_author_id = paper_id_to_author.get(node['id'], '')
            true_author_name = train_data.get(true_author_id, {}).get('name', '')
            
            db.merge_properties(
                type=NodeType.PUBLICATION, 
                node_id=node['id'], 
                properties={'true_author_id': true_author_id, 'true_author_name': true_author_name}
            )
            pbar.update(1)

"""
        for pub_id in values['normal_data']:
            props.append({'id': pub_id, 'properties': {'true_author': author_name}})
        pbar.update(1)
        if len(props) > 1000:
            db.merge_properties_batch(NodeType.PUBLICATION, props)
            props.clear()

    if props:
        db.merge_properties_batch(NodeType.PUBLICATION, props)
"""

Merging WhoIsWho train_author.json:   0%|          | 0/10829 [00:00<?, ?it/s]

"\n        for pub_id in values['normal_data']:\n            props.append({'id': pub_id, 'properties': {'true_author': author_name}})\n        pbar.update(1)\n        if len(props) > 1000:\n            db.merge_properties_batch(NodeType.PUBLICATION, props)\n            props.clear()\n\n    if props:\n        db.merge_properties_batch(NodeType.PUBLICATION, props)\n"

## Link nodes based on co-author relationships

In [7]:
def link_co_author_network(db: DatabaseWrapper, node_type: NodeType):
    num_nodes = db.count_nodes(node_type)
    attrs = ['id']
    co_author_map = defaultdict(list)
    co_author_overlap = defaultdict(int)
        
    print(f"Linking {node_type.value} nodes based on co-authorship ...")
    with tqdm(total=num_nodes, desc=f"Progress {node_type.value} co-authorship") as pbar:
        for nodes in db.iter_nodes(node_type, attrs):
            for node in nodes:
                co_authors = [author["name"] for author in data[node['id']]['authors']]
                for author in co_authors:
                    name = author.strip()
                    name = re.sub(r'[^A-Za-z\s]', '', name)
                    name_parts = name.split()
                    if len(name_parts) == 0:
                        continue
                    
                    surname = name_parts[-1]
                    given_name_initial = (name_parts[0] if len(name_parts) > 1 else ' ')[0]
                    abbrev = f"{surname} {given_name_initial}"
                    co_author_map[abbrev].append(node['id'])
                pbar.update(1)
    
    for k, v in co_author_map.items():
        for i in range(len(v)):
            for j in range(i + 1, len(v)):
                co_author_overlap[(v[i], v[j])] += 1
                
    for k, v in co_author_overlap.items():
        total_num_authors = data[k[0]]["authors"] + data[k[1]]["authors"]
        co_author_overlap[k] = v / len(total_num_authors)
    
    print(f"Max. co-authors: {max(len(v) for v in co_author_map.values())}")
    print(f"Max. co-author overlap: {max(co_author_overlap.values())}")
    
    print("Number of co-author pairs:", len(co_author_overlap))
    print("Number of co-author pairs with overlap > 0.25:", len([v for v in co_author_overlap.values() if v > 0.25]))
    
    print("Merging edges ...")
    edges_to_merge = [[k[0], k[1], {'sim': v}] for k, v in co_author_overlap.items() if v > 0.25]
    with tqdm(total=len(edges_to_merge), desc="Merging co-author edges") as pbar:
        for i in range(0, len(edges_to_merge), 1000):
            db.merge_edges(start_label=node_type, end_label=node_type, edge_type=EdgeType.SIM_AUTHOR, edges=edges_to_merge[i:i+1000])
            pbar.update(1000)

In [8]:
link_co_author_network(db, NodeType.PUBLICATION)

Linking Publication nodes based on co-authorship ...


Progress Publication co-authorship:   0%|          | 0/10829 [00:00<?, ?it/s]

Max. co-authors: 1649
Max. co-author overlap: 1.2181571815718157
Number of co-author pairs: 2840769
Number of co-author pairs with overlap > 0.25: 67731
Merging edges ...


Merging co-author edges:   0%|          | 0/67731 [00:00<?, ?it/s]

## Link nodes based on cosine similarity of their embeddings

In [20]:
def link_node_attr_cosine(db: DatabaseWrapper, node_type: NodeType, vec_attr: str, edge_type: EdgeType, threshold: float = 0.7, filter_empty_original_attr: str = None):
    num_nodes = db.count_nodes(node_type)
    edges = []
    attrs = ['id', vec_attr]
    if filter_empty_original_attr:
        attrs.append(filter_empty_original_attr)
        
    print(f"Linking {node_type.value} nodes based on {vec_attr} attribute ...")
    with tqdm(total=num_nodes, desc=f"Progress {node_type.value} {vec_attr}") as pbar:
        for nodes in db.iter_nodes(node_type, attrs):
            for node in nodes:
                if filter_empty_original_attr and not node[filter_empty_original_attr]:
                    pbar.update(1)
                    print(f"Skipping node {node['id']} because {filter_empty_original_attr} is empty")
                    continue
                k = 8
                similar_nodes = db.get_similar_nodes_vec(
                    node_type,
                    vec_attr,
                    node[vec_attr],
                    threshold,
                    k
                )
                for ix, row in similar_nodes.iterrows():
                    if row['id'] == node['id']:
                        continue
                    edges.append([node['id'], row['id']])
                    #db.merge_edge(node_type, node['id'], node_type, row['id'], edge_type, {"sim": row['sim']})
                if len(edges) > 1000:
                    print(f"Merging {len(edges)} edges ...")
                    db.merge_edges(start_label=node_type, end_label=node_type, edge_type=edge_type, edges=edges)
                    edges.clear()
                    
                pbar.update(1)
    if edges:
        db.merge_edges(start_label=node_type, end_label=node_type, edge_type=edge_type, edges=edges)

In [21]:
model_dim = model.get_sentence_embedding_dimension()
db.create_vector_index('title_index', NodeType.PUBLICATION, 'title_emb', model_dim)
db.create_vector_index('abstract_index', NodeType.PUBLICATION, 'abstract_emb', model_dim)
db.create_vector_index('venue_index', NodeType.PUBLICATION, 'venue_emb', model_dim)
link_node_attr_cosine(db, NodeType.PUBLICATION, 'title_emb', EdgeType.SIM_TITLE)
link_node_attr_cosine(db, NodeType.PUBLICATION, 'abstract_emb', EdgeType.SIM_ABSTRACT)
link_node_attr_cosine(db, NodeType.PUBLICATION, 'venue_emb', EdgeType.SIM_VENUE, threshold=0.9)

Linking Publication nodes based on title_emb attribute ...


Progress Publication title_emb:   0%|          | 0/10829 [00:00<?, ?it/s]

Merging 1004 edges ...
Merging 1008 edges ...
Merging 1003 edges ...
Merging 1002 edges ...
Merging 1001 edges ...
Merging 1005 edges ...
Merging 1004 edges ...
Merging 1001 edges ...
Merging 1001 edges ...
Merging 1007 edges ...
Merging 1001 edges ...
Merging 1002 edges ...
Merging 1007 edges ...
Merging 1002 edges ...
Merging 1008 edges ...
Merging 1001 edges ...
Merging 1001 edges ...
Linking Publication nodes based on abstract_emb attribute ...


Progress Publication abstract_emb:   0%|          | 0/10829 [00:00<?, ?it/s]

Merging 1001 edges ...
Merging 1003 edges ...
Merging 1004 edges ...
Merging 1008 edges ...
Merging 1003 edges ...
Merging 1007 edges ...
Merging 1003 edges ...
Merging 1008 edges ...
Merging 1004 edges ...
Merging 1001 edges ...
Merging 1008 edges ...
Merging 1008 edges ...
Merging 1004 edges ...
Merging 1003 edges ...
Merging 1002 edges ...
Merging 1001 edges ...
Merging 1004 edges ...
Merging 1002 edges ...
Merging 1002 edges ...
Merging 1001 edges ...
Merging 1008 edges ...
Merging 1008 edges ...
Merging 1001 edges ...
Merging 1006 edges ...
Merging 1005 edges ...
Merging 1004 edges ...
Merging 1001 edges ...
Merging 1002 edges ...
Merging 1002 edges ...
Merging 1004 edges ...
Merging 1008 edges ...
Merging 1003 edges ...
Merging 1001 edges ...
Merging 1003 edges ...
Merging 1005 edges ...
Merging 1006 edges ...
Merging 1002 edges ...
Merging 1008 edges ...
Linking Publication nodes based on venue_emb attribute ...


Progress Publication venue_emb:   0%|          | 0/10829 [00:00<?, ?it/s]

Merging 1001 edges ...
Merging 1005 edges ...
Merging 1004 edges ...
Merging 1002 edges ...
Merging 1003 edges ...
Merging 1002 edges ...
Merging 1003 edges ...
Merging 1004 edges ...
Merging 1007 edges ...
Merging 1004 edges ...
Merging 1004 edges ...
Merging 1006 edges ...
Merging 1003 edges ...
Merging 1001 edges ...
Merging 1002 edges ...
Merging 1002 edges ...
Merging 1004 edges ...
Merging 1002 edges ...
Merging 1001 edges ...
Merging 1007 edges ...
Merging 1007 edges ...
Merging 1006 edges ...
Merging 1003 edges ...
Merging 1002 edges ...
Merging 1007 edges ...
Merging 1003 edges ...
Merging 1007 edges ...
Merging 1006 edges ...
Merging 1005 edges ...
Merging 1008 edges ...
Merging 1002 edges ...
Merging 1001 edges ...
Merging 1006 edges ...
Merging 1003 edges ...
Merging 1003 edges ...
Merging 1003 edges ...
Merging 1001 edges ...
Merging 1008 edges ...
Merging 1004 edges ...
Merging 1006 edges ...
Merging 1004 edges ...
Merging 1006 edges ...
Merging 1005 edges ...
Merging 100

In [4]:
# Remove edges if according attribute is empty
db.delete_edges_for_empty_attr(NodeType.PUBLICATION, EdgeType.SIM_TITLE, 'title')
db.delete_edges_for_empty_attr(NodeType.PUBLICATION, EdgeType.SIM_ABSTRACT, 'abstract')
db.delete_edges_for_empty_attr(NodeType.PUBLICATION, EdgeType.SIM_VENUE, 'venue')

In [22]:
print(db.count_nodes(NodeType.PUBLICATION))

10829


In [18]:
gs = GraphSampling(
    node_spec=[NodeType.PUBLICATION], 
    edge_spec=[], 
    node_properties=['abstract_emb', 'title_emb', 'venue_emb']
)

In [19]:
nodes = gs.random_nodes(
    node_type=NodeType.PUBLICATION,
    node_properties=['abstract_emb', 'title_emb', 'venue_emb', 'true_author'],
    n=1000
)

for node in nodes:
    print(node)
    break

{'id': '6IsfnuWU', 'abstract_emb': None, 'title_emb': None, 'venue_emb': None, 'true_author': None}
